## Day 51 Proyect: Internet Complaint Twitter Bot

### Proyecto Día 51 – Bot para Reclamar Velocidad de Internet 🐦📉

Este proyecto automatiza la ejecución de un test de velocidad desde Speedtest.net y si la velocidad está por debajo de la prometida, publica un tweet de reclamo.

📌 Se implementa utilizando programación orientada a objetos (POO) para encapsular responsabilidades:

- `InternetSpeedTwitterBot`: clase principal que realiza el scraping y publica el tweet.

📌 Herramientas utilizadas:
- Selenium + undetected_chromedriver para evitar bloqueos
- Interacción con formularios, botones y tiempo de espera dinámico
- Manejo de autenticación segura


![SpeedTracker](./SpeedTracker_v1.png)

*SpeedTracker*

![Twitter Message](./Twitter_v1.png)

*Twitter Message*

In [1]:
from selenium.webdriver.common.by import By                 # Libreria para seleccionar elementos
from selenium.webdriver.common.keys import Keys             # Libreria para enviar acciones de teclado
import undetected_chromedriver as uc                        # Libreria para evitar detección de Selenium
from selenium.webdriver.support.ui import WebDriverWait     # Libreria para esperar a que se carguen los elementos
from selenium.webdriver.support import expected_conditions as EC    # Libreria para esperar a que se cumplan ciertas condiciones
import time                                                 # Libreria para manejar el tiempo
import os                                                   # Libreria para manejar el sistema operativo
from dotenv import load_dotenv                              # Libreria para cargar variables de entorno

In [2]:
# Carga las variables del archivo .env
load_dotenv()  

True

#### 🧱 Clase principal: InternetSpeedTwitterBot

La clase contiene atributos como velocidad prometida, credenciales y métodos para:
- Medir velocidad (`get_internet_speed`)
- Publicar en Twitter (`tweet_at_provider`)
- Cerrar el navegador (`close_browser`)

In [3]:
class InternetSpeedTwitterBot:
    def __init__(self, promised_down, promised_up, twitter_email, twitter_password, twitter_username):
        """🔧 Constructor de la clase InternetSpeedTwitterBot"""
        # 📊 Almacenamos los valores de velocidades prometidas por el proveedor
        self.promised_down = promised_down                  # Velocidad prometida de descarga
        self.promised_up = promised_up                      # Velocidad prometida de subida
        
        # 🔐 Guardamos las credenciales de acceso a Twitter
        self.twitter_email = twitter_email                  # Correo de Twitter
        self.twitter_password = twitter_password            # Contraseña de Twitter
        self.twitter_username = twitter_username            # Nombre de usuario de Twitter
        
        # 📈 Inicializamos variables para almacenar resultados del test
        self.download = 0
        self.upload = 0                                 

        # 🔧 Configurar driver con opciones para evitar detección de automatización
        options = uc.ChromeOptions()                                           # Creamos una instancia de ChromeOptions para undetected_chromedriver
        options.add_argument("--no-first-run")                                 # Prevenir configuraciones iniciales
        # options.add_argument("--start-maximized")                              # Opcional: abrir maximizado
        options.add_argument("--no-default-browser-check")                     # Prevenir chequeo de navegador por defecto
        options.add_argument("--disable-blink-features=AutomationControlled")  # Evitar detección de Selenium
        options.add_argument("--disable-infobars")                             # Desactivar la barra de información
        self.driver = uc.Chrome(options=options)                               # Inicializamos el driver con las opciones configuradas

        # ⏱️ Configuramos WebDriverWait para esperas explícitas (hasta 10 segundos)
        self.wait = WebDriverWait(self.driver, 10)

    def get_internet_speed(self):
        """🚀 Método para realizar la prueba de velocidad en speedtest.net"""
        # Navegar a la página de Speedtest
        self.driver.get("https://www.speedtest.net/")

        # ▶️ Iniciamos la prueba de velocidad
        start_btn = self.wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "start-text")))  # Localizamos el botón de Go
        start_btn.click()                                                                  # Hacemos clic para iniciar la prueba
        print("⏳ Ejecutando prueba de velocidad...")
        time.sleep(45)                                                                     # ⌛ Tiempo mínimo estimado para completar la prueba

        # Esperar a que aparezcan los valores de descarga y subida
        download_elem = self.wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "download-speed"))) # Localizamos el elemento de velocidad de descarga
        upload_elem = self.wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "upload-speed")))     # Localizamos el elemento de velocidad de subida
        
        # 📊 Obtenemos los resultados de la prueba
        self.download = float(download_elem.text)
        self.upload = float(upload_elem.text)
        print(f"⬇️ Download: {self.download} Mbps")
        print(f"⬆️ Upload:   {self.upload} Mbps\n")
        return self.download, self.upload

    def tweet_at_provider(self):
        """📣 Método para publicar un tweet de reclamo al proveedor de internet"""
        # 🔄 Navegamos al nuevo flujo de inicio de sesión de Twitter
        self.driver.get("https://x.com/i/flow/login")

        # 🔑 Esperamos a que el campo de correo esté listo y lo completamos
        email_input = self.wait.until(EC.element_to_be_clickable((By.NAME, "text")))
        email_input.send_keys(self.twitter_email)
        email_input.send_keys(Keys.ENTER)

        # ⚠️ Esperamos a que el campo de Telefono/Username por doble verificación esté listo y lo completamos
        password_input = self.wait.until(EC.element_to_be_clickable((By.NAME, "text")))
        password_input.send_keys(self.twitter_username)
        password_input.send_keys(Keys.ENTER)

        # 🔒 Esperamos a que el campo de contraseña esté listo y lo completamos
        password_input = self.wait.until(EC.element_to_be_clickable((By.NAME, "password")))
        password_input.send_keys(self.twitter_password)
        password_input.send_keys(Keys.ENTER)
        time.sleep(7)

        # 📝 Preparamos el mensaje de reclamo con datos reales
        mensaje = (
            f"Hola, mi proveedor me prometió {self.promised_down}Mbps de descarga y "
            f"{self.promised_up}Mbps de subida, pero estoy recibiendo "
            f"{self.download}Mbps / {self.upload}Mbps. ¿¡Qué pasa!? #InternetLento"
        )

        # ✏️ Esperamos a que el cuadro de texto del tweet esté presente y escribimos el mensaje
        tweet_box = self.wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".public-DraftStyleDefault-block.public-DraftStyleDefault-ltr"))
        )
        tweet_box.send_keys(mensaje)

        # 🚀 Esperamos a que el botón de publicar esté clickeable y enviamos el tweet
        tweet_button = self.wait.until(EC.element_to_be_clickable((By.XPATH,"//span[text()='Postear']/ancestor::button")))
        #tweet_button.click()
        print(f"📢 Tweet publicado:\n{mensaje}")

    def close_browser(self):
        """🔒 Método para cerrar el navegador y liberar recursos"""
        self.driver.quit()  # Cerramos todas las ventanas y finalizamos el proceso


#### 🟩 Ejecución del Programa

Con la clase `InternetSpeedTwitterBot` ya definida, podemos crear una instancia del bot, medir la velocidad de internet, y si está por debajo del umbral prometido, se publicará automáticamente un tweet.

📌 El flujo es el siguiente:
1. Inicializa el navegador con Selenium sin ser detectado.
2. Accede a Speedtest.net y mide velocidad de descarga y subida.
3. Si la velocidad está por debajo de lo prometido, abre Twitter y publica una queja.
4. Cierra el navegador al finalizar.

In [5]:
# Datos de autenticación 
TWITTER_EMAIL = os.environ["TWITTER_EMAIL"]
TWITTER_PASSWORD = os.environ["TWITTER_PASSWORD"]
TWITTER_USERNAME = os.environ["TWITTER_USERNAME"]

In [6]:
# Variables de validación
VELOCIDAD_PROM_DESCARGA = 50                  # Velocidad prometida de descarga
VELOCIDAD_PROM_SUBIDA = 50                    # Velocidad prometida de subida

# Creamos una instancia de la clase
bot = InternetSpeedTwitterBot(promised_down=VELOCIDAD_PROM_DESCARGA
                            , promised_up=VELOCIDAD_PROM_SUBIDA
                            , twitter_email=TWITTER_EMAIL
                            , twitter_password=TWITTER_PASSWORD
                            , twitter_username=TWITTER_USERNAME
)  

bot.get_internet_speed()  # Ejecutamos la prueba de velocidad
bot.tweet_at_provider()  # Publicamos el tweet


⏳ Ejecutando prueba de velocidad...
⬇️ Download: 6.8 Mbps
⬆️ Upload:   4.93 Mbps

📢 Tweet publicado:
Hola, mi proveedor me prometió 50Mbps de descarga y 50Mbps de subida, pero estoy recibiendo 6.8Mbps / 4.93Mbps. ¿¡Qué pasa!? #InternetLento


#### 🧠 Conclusión

Este proyecto demuestra cómo utilizar Python y Selenium para automatizar tareas reales del día a día.  
Al integrar una herramienta como Speedtest con redes sociales, logramos:

- Automatizar la medición de velocidad de internet con scraping visual.
- Generar una alerta pública (tweet) basada en condiciones lógicas.
- Controlar interfaces gráficas sin intervención humana.

Este tipo de automatización puede adaptarse a muchos otros contextos: reclamos automáticos, seguimiento de servicios, pruebas de conexión programadas, entre otros.